In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [13]:
seed = 42
iteration = 3000
threshold = 0.055

df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("submission.csv").drop(columns=['id'])

df_all = pd.concat([df_train, df_test]).reset_index(drop=True)

In [14]:
# ver_win_rate_x

x=2
print(f"|{df_all['ver_win_rate_x'].unique()[x]}|{df_all[(df_all['ver_win_rate_x']==df_all['ver_win_rate_x'].unique()[x])]['business_area'].unique()[0]}|{len(df_all[(df_all['ver_win_rate_x']==df_all['ver_win_rate_x'].unique()[x])])}|")
print(df_all[(df_all['ver_win_rate_x']==df_all['ver_win_rate_x'].unique()[x])]['business_area'].unique())
df_all[(df_all['ver_win_rate_x']==df_all['ver_win_rate_x'].unique()[x])].loc[:,['ver_win_rate_x', 'business_area']]

|0.0005719551277132|education|2039|
['education']


,ver_win_rate_x,business_area
12336,0.000572,education
19237,0.000572,education
22705,0.000572,education
22706,0.000572,education
22707,0.000572,education
...,...,...
64370,0.000572,education
64414,0.000572,education
64461,0.000572,education
64485,0.000572,education


In [15]:
# ver_win_ratio_per_bu

x=23

print(f"|{df_all['ver_win_ratio_per_bu'].unique()[x]}|{df_all[(df_all['ver_win_ratio_per_bu']==df_all['ver_win_ratio_per_bu'].unique()[x])]['business_unit'].unique()[0]}|{df_all[(df_all['ver_win_ratio_per_bu']==df_all['ver_win_ratio_per_bu'].unique()[x])]['business_area'].unique()[0]}|{len(df_all[(df_all['ver_win_ratio_per_bu']==df_all['ver_win_ratio_per_bu'].unique()[x])])}|")
print(df_all[(df_all['ver_win_ratio_per_bu']==df_all['ver_win_ratio_per_bu'].unique()[x])]['business_area'].unique())
df_all[(df_all['ver_win_ratio_per_bu']==df_all['ver_win_ratio_per_bu'].unique()[x])].loc[:,['ver_win_ratio_per_bu', 'business_unit', 'business_area']]

|0.0287769784172661|AS|public facility|209|
['public facility']


,ver_win_ratio_per_bu,business_unit,business_area
29204,0.028777,AS,public facility
29205,0.028777,AS,public facility
29206,0.028777,AS,public facility
29207,0.028777,AS,public facility
29208,0.028777,AS,public facility
...,...,...,...
62897,0.028777,AS,public facility
63887,0.028777,AS,public facility
64053,0.028777,AS,public facility
64078,0.028777,AS,public facility


In [16]:
def new_ratio(df: pd.DataFrame):
    solution = {
        'corporate / office':{'AS': 0.0268456375838926, 'ID': 0.0645661157024793, 'Solution': 0.0344827586206896},
        'education':{'ID': 0.048629531388152, 'AS': 0.0514705882352941},
        'special purpose':{'ID': 0.0640703517587939, 'AS': 0.022633744855967},
        'hospital & health care':{'ID': 0.1311475409836065, 'AS': 0.1285714285714285},
        'residential (home)':{'ID': 0.0354838709677419, 'AS': 0.0201207243460764},
        'government department':{'ID': 0.0794117647058823, 'AS': 0.0227272727272727},
        'retail':{'ID': 0.0498402555910543, 'AS': 0.0115830115830115},
        'hotel & accommodation':{'ID': 0.071345029239766},
        'factory':{'ID': 0.0369127516778523, 'AS': 0.0609243697478991},
        'power plant / renewable energy':{'ID': 0.2857142857142857, 'AS': 0.2272727272727272},
        'transportation':{'ID': 0.0535714285714285},
        'public facility':{'ID': 0.031578947368421, 'AS': 0.0287769784172661}
    }
    
    for business_area in solution.keys():
        for business_unit in solution[business_area].keys():
            df.loc[((df['business_area']==business_area) & (df['business_unit']==business_unit)), ['new_ver_win_ratio_per_bu']]=solution[business_area][business_unit]
    
    df = df.drop(columns='ver_win_ratio_per_bu')
    return df

df_all = new_ratio(df_all)
df_all

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,business_area,business_subarea,lead_owner,is_converted,new_ver_win_ratio_per_bu
0,1.00,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,corporate / office,Engineering,0,True,0.026846
1,1.00,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,corporate / office,Advertising,1,True,0.026846
2,1.00,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,corporate / office,Construction,2,True,0.026846
3,1.00,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,corporate / office,IT/Software,3,True,0.026846
4,1.00,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.003079,corporate / office,NaN,4,True,0.026846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64565,0.50,/São Paulo/Brazil,AS,NaN,40292,NaN,Enterprise,10.0,NaN,NaN,...,LGESP,NaN,0,0,NaN,NaN,NaN,97,0,NaN
64566,0.25,General / / United States,IT,NaN,47466,NaN,Enterprise,0.0,NaN,NaN,...,LGEUS,NaN,0,0,NaN,NaN,NaN,438,1,NaN
64567,0.75,/ OURO BRANCO / Brazil,AS,NaN,46227,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGESP,less than 3 months,0,0,NaN,NaN,NaN,97,0,NaN
64568,0.00,/ / Germany,IT,NaN,45667,End Customer,SMB,NaN,NaN,NaN,...,LGEDG,NaN,0,0,NaN,NaN,NaN,429,0,NaN


In [17]:
for idx, value in enumerate(df_all['customer_country']):
    if type(value) != float:
        new_value = value.split('/')[-1].replace(' ', '').lower()
        if ('usa' in new_value) or ('unitedstates' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        elif new_value == '':
            df_all.loc[idx, ['customer_country']]=np.nan
        else:
            df_all.loc[idx, ['customer_country']] = new_value
        
        if ('mumbai' in new_value):
            df_all.loc[idx, ['customer_country']]='india'
        if ('gujarat' in new_value):
            df_all.loc[idx, ['customer_country']]='india'
        if ('telangana' in new_value):
            df_all.loc[idx, ['customer_country']]='india'
        
        if ('fl' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('nevada' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('85wmainstsuitec,canton,ga30114,' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('230highlandave,suite531somervillema2143,' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('ironhorsecustomsllc4443genellawaynorthlasvegas,nv89031,' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('5301stevenscreekblvdsantaclaraca95051' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('7700westsunriseblvdplantationfl33322' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('mo64108.' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('3440rockefellerctwaldorf,md20602' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('ca91915-6002' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('2877prospectrd,fortlauderdale,fl33309' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('9landsdownestreetbostonma2215' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('160gouldstste300,needhamheights,massachusetts02494needhamma2721' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('oneconstitutionroadbostonma2129' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('77massachusettsavecambridgema2139' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('252beechavenuemelrosema2176' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('77massachusettsavecambridgema2139' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('36marginstpeabodyma1960' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('400centrestreetnewtonma2458' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('1755northbrownrd.suite200lawrenceville,ga30043' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('6kimballlanelynnfieldma1940' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('1275sistergroverdvanalstyne,tx75495  ' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('3nassonavenue' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('810nkingstondrpeoria,il61604-2145' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('mi48827' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('45n200wwillardut84340' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('1385nweberrd,romeoville,il60446,' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('ma02780' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('152bowdoinstreet' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('9820huntersvillenc28078' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('3000montourchurchroad' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('1380enterprisedr' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('6601carrollhighlandsrd' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('275mishawumroad' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('222maxinedr' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('2900highway280suite250birminghamal35223' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('il60069' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('100vestaviaparkwaybirminghamal35216' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('1100itbprovout84602' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('us' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        if ('700patrooncreekblvdalbanyny12206' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedstates'
        
        if ('caymanislands' in new_value) or ('britishvirginislands' in new_value) or ('bermuda' in new_value) or ('anguilla' in new_value):
            df_all.loc[idx, ['customer_country']]='unitedkingdom'
        
        if ('hanoi' in new_value):
            df_all.loc[idx, ['customer_country']]='vietnam'

        if ('turkey' in new_value):
            df_all.loc[idx, ['customer_country']]='türkiye'
        
        if ('1605ave.poncedeleón,suite400sanjuan,00909,puertorico' in new_value):
            df_all.loc[idx, ['customer_country']]='puertorico'
            
        if ('riodejaneiro' in new_value):
            df_all.loc[idx, ['customer_country']]='brazil'
        

country = df_all['customer_country'].value_counts()
train_country = df_all[:len(df_train)]['customer_country'].value_counts()

ban_list = country[country==1].index

df_all.loc[(df_all['customer_country'].isin(ban_list)), ['customer_country']]=np.nan

df_all

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,business_area,business_subarea,lead_owner,is_converted,new_ver_win_ratio_per_bu
0,1.00,philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,corporate / office,Engineering,0,True,0.026846
1,1.00,philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,corporate / office,Advertising,1,True,0.026846
2,1.00,india,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,corporate / office,Construction,2,True,0.026846
3,1.00,india,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,corporate / office,IT/Software,3,True,0.026846
4,1.00,india,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.003079,corporate / office,NaN,4,True,0.026846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64565,0.50,brazil,AS,NaN,40292,NaN,Enterprise,10.0,NaN,NaN,...,LGESP,NaN,0,0,NaN,NaN,NaN,97,0,NaN
64566,0.25,unitedstates,IT,NaN,47466,NaN,Enterprise,0.0,NaN,NaN,...,LGEUS,NaN,0,0,NaN,NaN,NaN,438,1,NaN
64567,0.75,brazil,AS,NaN,46227,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGESP,less than 3 months,0,0,NaN,NaN,NaN,97,0,NaN
64568,0.00,germany,IT,NaN,45667,End Customer,SMB,NaN,NaN,NaN,...,LGEDG,NaN,0,0,NaN,NaN,NaN,429,0,NaN


In [18]:
choice = [0.0037878787878787, 0.0039370078740157, 0.0118577075098814, 0.0135135135135135, 0.0151515151515151, 0.0175438596491228, 0.0181818181818181, 0.0199004975124378, 0.0311958405545927, 0.032258064516129, 0.0327868852459016, 0.0330578512396694, 0.0408163265306122, 0.043103448275862, 0.0434782608695652, 0.0446428571428571, 0.0485436893203883, 0.0575342465753424, 0.0666666666666666, 0.0888888888888888, 0.004, 0.0109890109890109, 0.0169491525423728, 0.0172413793103448, 0.0196078431372549, 0.0202020202020202, 0.0227272727272727, 0.025, 0.0289256198347107, 0.0289855072463768, 0.036036036036036, 0.037037037037037, 0.04, 0.0416666666666666, 0.0422535211267605, 0.0476190476190476, 0.0491803278688524, 0.0496894409937888, 0.0531914893617021, 0.0538922155688622, 0.0544217687074829, 0.0555555555555555, 0.0677966101694915, 0.0681818181818181, 0.0695652173913043, 0.0714285714285714, 0.0732484076433121, 0.0749486652977412, 0.075, 0.0806916426512968, 0.0833333333333333, 0.0843373493975903, 0.0869565217391304, 0.1052631578947368, 0.1136363636363636, 0.1162790697674418, 0.1184210526315789, 0.1186440677966101, 0.1241217798594847, 0.125, 0.1363636363636363, 0.1470588235294117, 0.1666666666666666, 0.1818181818181818, 0.2, 0.2142857142857142, 0.2307692307692307, 0.25, 0.2692307692307692, 0.3333333333333333, 0.3636363636363636, 0.3902439024390244, 0.4, 0.4444444444444444, 0.4615384615384615, 0.5, 0.6153846153846154, 0.6428571428571429, 0.8333333333333334, 1.0]

for locate in range(0,80,1):
       where = df_all.fillna('0').loc[(df_all['com_reg_ver_win_rate']==choice[locate]), ['com_reg_ver_win_rate', 'customer_country', 'business_unit', 'business_area']]
       print(f"|{choice[locate]}|{where['business_unit'].unique()[0]}|{where['business_area'].unique()[0]}|{[ k for k in sorted(where['customer_country'].unique())]}|")


|0.0037878787878787|AS|residential (home)|['0', 'argentina', 'bolivia', 'brazil', 'chile', 'colombia', 'dominicanrepublic', 'elsalvador', 'honduras', 'mexico', 'nicaragua', 'panama', 'peru', 'stkitts', 'trinidadandtobago', 'unitedkingdom', 'uruguay']|
|0.0039370078740157|AS|corporate / office|['0', 'antigua', 'argentina', 'bahamas', 'belize', 'brazil', 'chile', 'colombia', 'dominicanrepublic', 'elsalvador', 'guatemala', 'honduras', 'jamaica', 'mexico', 'panama', 'peru', 'uruguay']|
|0.0118577075098814|AS|special purpose|['0', 'argentina', 'brazil', 'chile', 'colombia', 'dominicanrepublic', 'ecuador', 'elsalvador', 'guatemala', 'jamaica', 'mexico', 'panama', 'paraguay', 'peru', 'unitedkingdom', 'uruguay']|
|0.0135135135135135|ID|special purpose|['0', 'albania', 'belgium', 'bosniaandherzegovina', 'bulgaria', 'croatia', 'czech', 'france', 'germany', 'greece', 'ireland', 'italy', 'latvia', 'netherlands', 'poland', 'portugal', 'romania', 'slovenia', 'spain', 'switzerland', 'unitedkingdom', 

In [19]:
df_all['customer_country'].value_counts().sort_values()[150:]

thailand           204
southafrica        220
singapore          267
ecuador            297
canada             303
france             326
indonesia          363
panama             366
hongkong           439
argentina          564
spain              576
türkiye            594
vietnam            673
egypt              726
poland             876
germany            963
italy             1262
chile             1335
peru              1346
saudiarabia       1572
unitedkingdom     1599
u.a.e             1750
colombia          2113
philippines       2858
mexico            2933
unitedstates      5292
brazil            9786
india            17761
Name: customer_country, dtype: int64